In [1]:
import openpyxl
from text_processing import readJSON, common_grams, bag, get_gram_id,remove_rows_with_none_elements, strip_text
from features import feat_cnt, constant
from sklearn.linear_model import LinearRegression

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from collections import defaultdict
import nltk
from sklearn.naive_bayes import BernoulliNB, GaussianNB, MultinomialNB, ComplementNB,CategoricalNB
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import PCA 
from sklearn.cluster import KMeans 
import matplotlib.pyplot as plt 
import numpy as np 
import pandas as pd 
from gensim.models import Word2Vec

In [2]:
def trunc(n, k):
    """
    Return the number truncated to k decimal places, e.g, k = 0 returns only the integer part
    """

    return int(n * 10**k) / 10**k

def measure(y_true, y_pred, k):
    """
    y_pred = binary vector
    y_true = binary vector

    Desc: Computes the accuracy, TP, TN, FP, FN, BER truncated to k decimal places
    """
    
    if len(y_pred) != len(y_true):
        print("Lengths not equal: y_pred={}, y_true={}".format(len(y_pred), len(y_true)))
        return

    TP, TN, FP, FN = 0, 0, 0, 0,
    for i in range(len(y_pred)):
        if y_pred[i] == y_true[i] and y_true[i] == 1:
            TP += 1
        elif y_pred[i] == y_true[i] and y_true[i] == 0:
            TN += 1
        elif y_pred[i] != y_true[i] and y_true[i] == 1:
            FN += 1
        elif y_pred[i] != y_true[i] and y_true[i] == 0:
            FP += 1 
    
    total = TP + TN + FP + FN
    print(total)
    FPR = TP / (TP + FP) if TP + FP > 0 else 0
    FPN = TN / (TN + FN) if TN + FN > 0 else 0
    acc = (TP + TN) / total
    precision = TP / (TP + FP) if TP + FP > 0 else 0
    recall = TP / (TP + FN) if TP + FN > 0 else 0
    f_score = 2 * precision * recall / (precision + recall) if (precision + recall) > 0 else 0
    BER = (FPR + FPN) / 2
    print(f_score, acc)
    return (TP, TN, FP, FN, trunc(precision, k), trunc(recall, k), trunc(acc, k), trunc(f_score, k))

def prod(S):
    product = 1
    for x in S:
        product *= x
    return product

In [3]:
# Open the Excel file
excel_file_path = 'Book1.xlsx'
workbook = openpyxl.load_workbook(excel_file_path)

# Select the active sheet (you can also select a specific sheet by name: workbook['Sheet1'])
sheet = workbook.active

# Create empty lists to store data from each column
columns = [[] for _ in range(sheet.max_column)]
counter = 0
# Iterate through each row and store data from each column
for row in sheet.iter_rows(values_only=True):
    counter +=1
    for i, cell_value in enumerate(row):
        
        columns[i].append(cell_value)


# Close the workbook
workbook.close()


In [4]:
columns = remove_rows_with_none_elements(columns)
train_data = []
train_data.append(columns[0][0:2587])
train_data.append(columns[1][0:2587])

test_data = []
test_data.append(columns[0][2587:])
test_data.append(columns[1][2587:])
counter = 0

train_texts = train_data[0]
top_unigrams = common_grams(train_texts, 1, 1000, no_stopwords=True)

uni_bag = bag(top_unigrams)
uni_id = get_gram_id(uni_bag)

print(top_unigrams)
print(uni_bag)
print(uni_id)

[(3355, 'one'), (3011, 'like'), (2465, 'time'), (2117, 'get'), (2063, 'would'), (1946, 'im'), (1759, 'know'), (1708, 'people'), (1644, 'new'), (1482, 'good'), (1471, 'really'), (1444, 'back'), (1434, 'much'), (1423, 'well'), (1419, 'even'), (1407, 'first'), (1404, 'think'), (1403, 'also'), (1392, 'us'), (1390, 'dont'), (1383, 'see'), (1357, 'way'), (1333, 'make'), (1285, 'day'), (1231, 'could'), (1187, 'go'), (1176, 'want'), (1143, 'many'), (1139, 'last'), (1115, 'life'), (1108, 'work'), (1101, 'little'), (1093, 'things'), (1088, 'two'), (1068, 'going'), (1025, 'something'), (1020, 'love'), (1017, 'years'), (1000, 'still'), (980, 'got'), (947, 'made'), (938, 'say'), (917, 'take'), (915, 'around'), (910, 'said'), (909, 'never'), (900, 'right'), (833, 'year'), (833, 'need'), (825, 'ive'), (806, 'great'), (792, 'world'), (779, 'every'), (759, 'always'), (744, 'blog'), (735, 'thing'), (735, 'may'), (734, 'long'), (720, 'best'), (711, 'since'), (701, 'look'), (700, 'feel'), (699, 'home'), (

In [5]:

X = [feat_cnt(t, uni_bag, uni_id, 1, no_stopwords=False) for t in train_texts]
X = constant([X])
y = [1 if d == 'M' else 0 for d in train_data[1]] 


print(y)
model = LogisticRegression(max_iter=1000)
model.fit(X, y)
model_b = LogisticRegression(max_iter=1000, class_weight="balanced")
model_b.fit(X, y)

test_texts = test_data[0]

X_test = [feat_cnt(t, uni_bag, uni_id, 1, no_stopwords=False) for t in test_texts]
X_test = constant([X_test])
y_test = [1 if d == 'M' else 0 for d in test_data[1]]

y_pred = model.predict(X_test)
y_pred_b = model_b.predict(X_test) 
print(X_test)
unbalanced_logistic = "Unbalanced: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred, 5))
balanced_logistic = "Balanced: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred_b, 5))

[1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 0, 0, 1, 1, 1, 

In [6]:

y_pred = []
blogs_test = test_data[0]
for r in blogs_test:

    r_grams = strip_text(r, n)
    
    filtered = [g for g in r_grams if g in grams]
    # Guess yes if there are no top_grams appearing in text
    if len(filtered) == 0:
        y_pred.append(1)
    # Compute according to naive probability model
    else:

        prob_label_positive = p_positive * prod([gram_positive_prob[g] for g in filtered])
        prob_label_negative = p_negative * prod([1-gram_positive_prob[g] for g in filtered])
        #(prob_label_positive, prob_label_negative )
        if prob_label_positive > prob_label_negative:
            y_pred.append(1)
        else:
            y_pred.append(0)

print("Baseline: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred, 5)))



NameError: name 'n' is not defined

In [2]:
# create vectorizer 
def cluster(dataset, num_clusters = 4, plot = False):
    vectorizer = TfidfVectorizer(stop_words='english') 
    
    # vectorizer the text documents 
    vectorized_documents = vectorizer.fit_transform(dataset) 
    print(vectorized_documents.get_shape)
    # reduce the dimensionality of the data using PCA 
    pca = PCA(n_components=2) 
    reduced_data = pca.fit_transform(vectorized_documents.toarray()) 

    kmeans = KMeans(n_clusters=num_clusters, n_init=5, 
                    max_iter=500, random_state=42) 
    kmeans.fit(vectorized_documents) 
    
    # create a dataframe to store the results 
    results = pd.DataFrame() 
    results['document'] = dataset
    results['cluster'] = kmeans.labels_ 
    
    # print the results
    
    
    # plot the results 
    if plot:
      colors = ['red', 'green', 'blue'] 
      cluster = ['cluster 1', 'cluster 2', 'cluster 3'] 
      for i in range(num_clusters): 
          plt.scatter(reduced_data[kmeans.labels_ == i, 0], 
                      reduced_data[kmeans.labels_ == i, 1],  
                      s=10, color=colors[i],  
                      label=f' {cluster[i]}') 
      plt.ylim(-0.1, 0.1)
      plt.xlim(-0.15, 0.2)
      plt.legend() 
      plt.show()
    
    cluster_set = []
    for i in range(len(dataset)):
        cluster_label = results.loc[i, 'cluster']
        cluster_number = cluster_label + 1
        cluster_set.append([cluster_number])
    # Get the cluster label assigned to the element
    

    # Find out which cluster the element belongs to
      # Adding 1 to match cluster numbering starting from 1
    return cluster_set
    

In [3]:
cluster_train_text = cluster(blog_train,3,True)
cluster_test_text = cluster(test_texts,3)


""" 
modelNB = MultinomialNB()
modelNB.fit(cluster_train_text, y)


y_pred = modelNB.predict(cluster_test_text)

model_MB = "Model: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred, 5)) """


NameError: name 'train_texts' is not defined

In [ ]:
modelNB = ComplementNB()
modelNB.fit(X, y)

print(len(X))
y_pred = modelNB.predict(X_test)

model_MB = "Model: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred, 5))

2587
639
0.5897920604914935 0.6604068857589984


In [ ]:
model_c = LogisticRegression(max_iter=1000)
model_c.fit(cluster_train_text, y)
model_b = LogisticRegression()
model_b.fit(cluster_train_text, y)
modelNB = CategoricalNB()
modelNB.fit(cluster_train_text, y)


y_pred = model_b.predict(cluster_test_text)
y_pred2 = modelNB.predict(cluster_test_text)

cluster_model_NB = "cluster_model_NB: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred2, 5))
cluster_model_regression = "cluster_model_regression: (TP, TN, FP, FN, precision, recall, acc, f_score) = {}".format(measure(y_test, y_pred, 5))


639
0.2127659574468085 0.5367762128325508
639
0.2127659574468085 0.5367762128325508


In [ ]:
print(unbalanced_logistic)
print(balanced_logistic)
print(model_MB)
print(cluster_model_regression)
print(cluster_model_NB)

Unbalanced: (TP, TN, FP, FN, precision, recall, acc, f_score) = (181, 221, 114, 123, 0.61355, 0.59539, 0.6291, 0.60434)
Balanced: (TP, TN, FP, FN, precision, recall, acc, f_score) = (183, 216, 119, 121, 0.60596, 0.60197, 0.62441, 0.60396)
Model: (TP, TN, FP, FN, precision, recall, acc, f_score) = (156, 266, 69, 148, 0.69333, 0.51315, 0.6604, 0.58979)
cluster_model_regression: (TP, TN, FP, FN, precision, recall, acc, f_score) = (40, 303, 32, 264, 0.55555, 0.13157, 0.53677, 0.21276)
cluster_model_NB: (TP, TN, FP, FN, precision, recall, acc, f_score) = (40, 303, 32, 264, 0.55555, 0.13157, 0.53677, 0.21276)


In [ ]:
for i in range(len(train_texts)):
    print(train_texts[i])
    print(columns[1][i])
    if i == 10:
        break

 Long time no see. Like always I was rewriting it from scratch a couple of times. But nevertheless it's still java and now it uses metropolis sampling to help that poor path tracing converge.

Btw. I did MLT on yesterday evening after 2 beers (it had to be Ballmer peak).

Altough the implementation is still very fresh it easily outperforms standard path tracing, what is to be seen especially when difficult caustics are involved.

I've implemented spectral rendering too, it was very easy actually, cause all computations on wavelengths are linear just like rgb. But then I realised that even if it does feel more physically correct to do so, whats the point? 3d applications are operating in rgb color space, and because I cant represent a rgb color as spectrum interchangeably I have to approximate it, so as long as I'm not running a physical simulation or something I don't see the benefits (please correct me if I'm wrong), thus I abandoned that.
M
 Guest Demo: Eric Iverson’s Itty Bitty Sear